In [1]:
import tensorflow as tf
from tensorflow import keras

##### 글로럿 -> 활성화 함수 없음, 하이퍼볼릭 탄젠트, 로지스틱, 소프트맥스
##### HE -> ReLU
##### 르쿤 -> SELU

In [3]:
# 케라스는 균등분포의 글로럿 초기화 사용
# He초기화 : fan_in -> ReLU
keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal') # he_uniform

In [4]:
# fan_in 대신 fan_avg기반의 균등분포 He 초기화 -> Variance Scaling
he_avg_init = keras.initializers.VarianceScaling(scale = 2., mode='fan_avg',
                                                distribution='uniform')
keras.layers.Dense(10, activation='sigmoid', kernel_initializer=he_avg_init)

In [7]:
# LeakyReLU : alpha=0.01 ->  적용하려는 층 뒤에 추가
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [8]:
# PReLU -> 적용하려는 층 뒤에 추가
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(), # LeakyReLU(alpha=0.2)
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [6]:
# elu
keras.layers.Dense(10, activation="elu")

In [5]:
# selu : 모두 완전 연결 층이고, 모든 은닉층이 SELU만 쓰면 자기정규화(평균0, 표준편차1)가 된다.
keras.layers.Dense(10, activation="selu",
                   kernel_initializer="lecun_normal")

In [9]:
# 배치 정규화 : 입력을 원점에 맞추고 정규화한 후, 각 층에서 두개의 새 파라미터로 결과값 스케일 조정하고 이동
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

# 3136 = 784 * 4 (감마, 베타, 뮤, 시그마)
# Non-trainable : 2368 = (3136+1200+400)/2

In [10]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (Batch  (None, 784)               3136      
 Normalization)                                                  
                                                                 
 dense_10 (Dense)            (None, 300)               235500    
                                                                 
 batch_normalization_1 (Bat  (None, 300)               1200      
 chNormalization)                                                
                                                                 
 dense_11 (Dense)            (None, 100)               30100     
                                                                 
 batch_normalization_2 (Bat  (None, 100)              

In [11]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [15]:
# 활성화 함수 전에 배치 정규화 층 추가하는 것이 좋음
# 은닉층에서 활성화함수 지정하지 않고, 배치 정규화 층 뒤에 별도의 층으로 추가해야
# 배치 정규화 층은 입력마다 이동 파라미터 포함하기에 이전 층에서 편향 뺌
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, kernel_initializer='he_normal', use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('elu'),
    keras.layers.Dense(100, kernel_initializer='he_normal', use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('elu'), 
    keras.layers.Dense(10, activation='softmax')
])

In [16]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization_3 (Bat  (None, 784)               3136      
 chNormalization)                                                
                                                                 
 dense_13 (Dense)            (None, 300)               235200    
                                                                 
 batch_normalization_4 (Bat  (None, 300)               1200      
 chNormalization)                                                
                                                                 
 activation (Activation)     (None, 300)               0         
                                                                 
 dense_14 (Dense)            (None, 100)              

In [17]:
# 그레이디언트 클리핑 : 역전파될 때 일정 임곗값 넘어서지 못하게 그레이디언트 잘라냄
optimizer = keras.optimizers.SGD(clipvalue=1.0) # clipnorm(그레이디언트 벡터 방향 바꾸지 못함)
model.compile(loss='mse', optimizer=optimizer)

전이 학습

In [2]:
import numpy as np

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [22]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [27]:
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation='sigmoid'))
# model_B_on_A를 훈련할 때 model_A도 영향 받음

In [30]:
# 영향받지 않으려면, 층 재사용하기 전에 model_A 클론하기
model_A_clone = keras.models.clone_model(model_A) # 구조 복제
model_A_clone.set_weights(model_A.get_weights()) # 가중치 복사(clone_model은 가중치 복사하지 않아 따로 해줘야)

model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation='sigmoid'))

In [31]:
# 새로운 출력층이 랜덤하게 초기화 -> 큰 오차 -> 큰 오차 그레이디언트가 재사용된 가중치 망침
# 처음 몇번의 에포크 동안 재사용된 층 동결하고, 새로운 층에게 적절한 가중치를 학습할 시간을 줌
for layer in model_B_on_A.layers[:-1] :
    layer.trainable=False  # 모든 레이어의 가중치가 훈련 불가능 -> 업데이트 안됨
    
# 층을 동결하거나 해제하면 다시 컴파일 해야함
model_B_on_A.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [32]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1] :
    layer.trainable = True
    
model_B_on_A.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 47ms/step - loss: 1.0223 - accuracy: 0.3650 - val_loss: 0.9017 - val_accuracy: 0.3915
Epoch 2/4
7/7 [==============================] - 0s 15ms/step - loss: 0.8608 - accuracy: 0.4050 - val_loss: 0.7831 - val_accuracy: 0.4899
Epoch 3/4
7/7 [==============================] - 0s 15ms/step - loss: 0.7636 - accuracy: 0.4900 - val_loss: 0.7072 - val_accuracy: 0.5862
Epoch 4/4
7/7 [==============================] - 0s 16ms/step - loss: 0.6994 - accuracy: 0.5600 - val_loss: 0.6603 - val_accuracy: 0.6481
Epoch 1/16
7/7 [==============================] - 1s 41ms/step - loss: 0.3864 - accuracy: 0.8050 - val_loss: 0.1678 - val_accuracy: 0.9726
Epoch 2/16
7/7 [==============================] - 0s 17ms/step - loss: 0.0963 - accuracy: 0.9950 - val_loss: 0.0980 - val_accuracy: 0.9797
Epoch 3/16
7/7 [==============================] - 0s 17ms/step - loss: 0.0599 - accuracy: 0.9950 - val_loss: 0.0735 - val_accuracy: 0.9868
Epoch 4/16
7/7 [===============

In [33]:
model_B_on_A.evaluate(X_test_B, y_test_B) # 전이학습이 아닌 처음부터 직접짠 코드로(재사용하지 않은 층)하는 것보다 좋은 성능

63/63 [==============================] - 0s 2ms/step - loss: 0.0246 - accuracy: 0.9930


[0.024575011804699898, 0.9929999709129333]

옵티마이저

In [34]:
# 모멘텀 최적화 : 지역 최적점을 건너뛰도록 한다
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

In [35]:
# 네스테로프 가속 경사 : 비용함수의 그레이디언트, 모멘텀의 올바른 방향으로
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

In [36]:
# AdaGrad : 가장 가파른 차원을 따라 그레이디언트 벡터의 스케일 감소
# 학습률 감소시키지만, 가파른 차원에 더 빠르게 감소(적응적 학습률)
# 학습률 튜닝하지 않아도 됨
# 너무 빨리느려져 전역 최적점에 수렴하지 못함, 너무 일찍 멈춰 심층 신경망에는 사용 X
optimizer = keras.optimizers.Adagrad(learning_rate=0.001) # default lr

In [37]:
# RMSProp : 가장 최근 반복에서 비롯된 그레이디언트만 누적
optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9) # default

In [38]:
# Adam(적응적 모멘트 추정) : 모멘텀 + RMSprop
# 학습률 튜닝하지 않아도 됨
# Adamax, Nadam
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

#### 학습률 스케줄링  
: 큰 학습률로 시작해 학습 속도가 느려질 때 학습률 낮추는 전략

In [42]:
# 거듭제곱 기반 스케줄링 : 갈수록 천천히 감소시킴
# lr = lr0 / (1 + steps / s)**c  (keras에서 c=1, s=1/decay)
# ex) 학습률/2 -> 학습률/3 -> 학습률/4 -> ...
optimizer = keras.optimizers.legacy.SGD(learning_rate=0.01, decay=1e-4) # step의 역수

In [43]:
# 지수기반 스케줄링 : 스텝마다 10배씩 줄어듦
# lr = lr0 * 0.1**(epoch / s)
def exponential_decay_fn(epoch) :
    return 0.01 * 0.1**(epoch/20)

In [45]:
def exponential_decay(lr0, s) :  # 초기 학습률
    def exponential_decay_fn(epoch) :
        return lr0 * 0.1**(epoch/s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.01, s=20)

lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
# 에포크 시작할 때마다 학습률 업데이트

In [1]:
def exponential_decay_fn(epoch, lr) : # 현재 학습률
    return lr * 0.1**(1/20)
# 초기 학습률에만 의존

In [2]:
# 구간별 고정 스케줄링 : 일정 횟수 에포크동안 일정한 학습률, 그다음 또 다른 횟수의 에포크 동안 작은 학습률
def piecewise_constant_fn(epoch) :
    if epoch < 5 :
        return 0.01
    elif epoch < 15 :
        return 0.005
    else : 
        return 0.001

In [5]:
def piecewise_constant(boundaries, values) :
    boundaries = np.array([0] + boundaries)  # [0, 5, 15]
    values = np.array(values)
    def piecewise_constant_fn(epoch) :
        return values[np.argmax(boundaries > epoch)-1]
    return piecewise_constant_fn

piecewise_constant_fn = piecewise_constant([5, 15], [0.01, 0.005, 0.001])

lr_scheduler = keras.callbacks.LearningRateScheduler(piecewise_constant_fn)

In [8]:
# 성능기반 스케줄링 : 매 스텝마다 검증오차 측정하고, 오차 줄지 않으면 람다배만큼 학습률 감소시킴
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
# 최상의 검증 손실이 다섯 번의 연속적인 에포크 동안 향상되지 않을 때마다 학습률에 0.5 곱함

In [12]:
# tf.keras : 위들 중 하나를 사용해 학습률 정의하고, 이 학습률을 옵티마이저에 전달
# 에포크가 아니라 매 스텝마다 학습률 업데이트
s = 20 * len(X_train) // 32  # 20번 에포크에 담긴 전체 스텝 수
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.SGD(learning_rate)

규제를 사용해 과대적합 피하기

In [3]:
# l1, l2 규제
# l2() : 각 스텝에서 호출되는 규제 객체 반환하고, 이 손실은 최종손실에 합산
layer = keras.layers.Dense(100, activation='elu',
                          kernel_initializer='he_normal',
                          kernel_regularizer=keras.regularizers.l2(0.01))
# l1 : keras.regularizers.l1()
# l1 + l2 : keras.regularizers.l1_l2()

In [17]:
# 네트워크 모든 은닉층은 동일한 활성화함수/초기화전략/규제 적용하기에 동일한 매개변수 값 반복
# 이는 코드를 읽기 어렵게 만들고 버그 만듬 -> 반복문 사용하도록 리팩터링함
from functools import partial  # 기본 매개변수 값 사용해 함수 호출 감쌈
RegularizedDense = partial(keras.layers.Dense,
                          activation='elu',
                          kernel_initializer='he_normal',
                          kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation='softmax', kernel_initializer='glorot_uniform')
])

In [15]:
# Dropout : 모든 입력 뉴런에 주의를 기울여야하므로 입력값의 작은 변화에 덜 민감해짐
# 보통 출력층을 제외한 맨 위의 층부터 세 번째 층까지에 적용
# 훈련 후 각 입력의 연결 가중치에 보존 확률(1-p) 곱해야한다.
# 훈련 후 드롭아웃 빼고 훈련 손실 평가해야함
# 과대적합 -> 드롭아웃 비율 늘림
# 층 크면 -> 드롭아웃 비율 늘림
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
])

In [20]:
# 알파 드롭아웃 : SELU 활성화 함수 기반으로 자기 정규화하는 네트워크 규제
# 입력의 평균과 표준편차 유지
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

In [21]:
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
n_epochs = 20
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/20
1719/1719 [==============================] - 11s 6ms/step - loss: 0.6718 - accuracy: 0.7569 - val_loss: 0.6327 - val_accuracy: 0.8404
Epoch 2/20
1719/1719 [==============================] - 9s 6ms/step - loss: 0.5581 - accuracy: 0.7935 - val_loss: 0.5859 - val_accuracy: 0.8450
Epoch 3/20
1719/1719 [==============================] - 9s 5ms/step - loss: 0.5281 - accuracy: 0.8044 - val_loss: 0.5407 - val_accuracy: 0.8504
Epoch 4/20
1719/1719 [==============================] - 9s 5ms/step - loss: 0.5080 - accuracy: 0.8115 - val_loss: 0.4888 - val_accuracy: 0.8516
Epoch 5/20
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4959 - accuracy: 0.8174 - val_loss: 0.4783 - val_accuracy: 0.8530
Epoch 6/20
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4782 - accuracy: 0.8242 - val_loss: 0.4757 - val_accuracy: 0.8518
Epoch 7/20
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4736 - accuracy: 0.8255 - val_loss: 0.4538 - val_accurac

In [22]:
# 몬테 카를로 드롭아웃 : 훈련된 모델을 재훈련하거나 전혀 수정하지 않고 성능 향상 가능
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

y_probas = np.stack([model(X_test_scaled, training=True) for sample in range(100)])
y_proba = y_probas.mean(axis=0)
# model(X) : model.predict(X)와 비슷하지만 텐서로 반환
# training=True : 드롭아웃 층 활성화
# X_test_scaled.shape : (10000, 28, 28)
# y_probas.shape : (100, 10000, 10) ; 테스트 세트 10000개 샘플과 10개의 클래스, 모델 호출100번

In [23]:
np.round(model.predict(X_test_scaled[:1]), 2)  # 드롭아웃 끄고 첫 번째 샘플의 모델 예측 => 9클래스에 속함

1/1 [==============================] - 0s 120ms/step


array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.99]],
      dtype=float32)

In [25]:
np.round(y_probas[:, :1], 2) # 드롭아웃 활성화한 첫 번째 샘플의 모델 예측 => 9를 선호하지만 5나 7도 약간 있음

array([[[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.73, 0.  , 0.26]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.03, 0.  , 0.17, 0.  , 0.81]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.06, 0.  , 0.48, 0.  , 0.46]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.12, 0.  , 0.87]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.13, 0.  , 0.08, 0.  , 0.79]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.35, 0.  , 0.64]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.19, 0.  , 0.57, 0.  , 0.25]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.65, 0.  , 0.35]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.6 , 0.  , 0.39]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.23, 0.  , 0.77]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.06, 0.  , 0.16, 0.  , 0.78]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.42, 0.  , 0.56]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.15, 0.  , 0

In [30]:
np.round(y_proba[:1], 2) # MC드롭아웃 => 9로 예측하지만, 확신 확률 낮음

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.05, 0.  , 0.28, 0.  , 0.67]],
      dtype=float32)

In [31]:
y_std = y_probas.std(axis=0) # 표준 분포
np.round(y_std[:1], 2)

# 많은 분산 존재

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.1 , 0.  , 0.26, 0.  , 0.27]],
      dtype=float32)

In [34]:
y_pred = np.argmax(y_proba, axis=1)
accuracy = np.sum(y_pred == y_test) / len(y_test)
accuracy

0.8647

In [35]:
# 맥스-노름 규제
keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal',
                  kernel_constraint = keras.constraints.max_norm(1.)) # 가중치 노름이 특정 값 이하가 되도록

# 연습문제

In [36]:
## 8) CIFAR10 이미지 데이터셋에 심층신경망 훈련하기
# a. 100개 뉴런을 가진 은닉층 20개로 심층 신경망 만들기 (He초기화와 ELU 활성화 함수 사용)
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
for _ in range(20) :
    model.add(keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'))

In [37]:
# b. Nadam 옵티마이저, 조기종료 사용해 훈련하기 
# 이 데이터셋은 10개의 클래스와 32X32 크기의 컬러이미지 60000개로 구성(훈련 50000)
model.add(keras.layers.Dense(10, activation='softmax'))

In [38]:
optimizer=keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [39]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

170498071/170498071 [==============================] - 315s 2us/step


In [40]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)

model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks = [early_stopping_cb, model_checkpoint_cb])
model.evaluate(X_valid, y_valid)

Epoch 1/100
1405/1407 [============================>.] - ETA: 0s - loss: 4.9122 - accuracy: 0.1482

C:\Users\knuyh\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1407/1407 [==============================] - 55s 21ms/step - loss: 4.9098 - accuracy: 0.1482 - val_loss: 2.1785 - val_accuracy: 0.2088
Epoch 2/100
1407/1407 [==============================] - 26s 19ms/step - loss: 2.0863 - accuracy: 0.2307 - val_loss: 1.9896 - val_accuracy: 0.2694
Epoch 3/100
1407/1407 [==============================] - 30s 22ms/step - loss: 1.9681 - accuracy: 0.2765 - val_loss: 1.9093 - val_accuracy: 0.2998
Epoch 4/100
1407/1407 [==============================] - 26s 19ms/step - loss: 1.8906 - accuracy: 0.3050 - val_loss: 1.8274 - val_accuracy: 0.3372
Epoch 5/100
1407/1407 [==============================] - 26s 18ms/step - loss: 1.8334 - accuracy: 0.3335 - val_loss: 1.7956 - val_accuracy: 0.3446
Epoch 6/100
1407/1407 [==============================] - 27s 19ms/step - loss: 1.7832 - accuracy: 0.3508 - val_loss: 1.7916 - val_accuracy: 0.3502
Epoch 7/100
1407/1407 [==============================] - 25s 18ms/step - loss: 1.7401 - accuracy: 0.3709 - val_loss: 1.7150 - val_

[1.5824220180511475, 0.4602000117301941]

In [44]:
# c. 배치정규화 추가해 학습 곡선 비교
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20) :
    model.add(keras.layers.Dense(100, kernel_initializer='he_normal'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('elu'))
model.add(keras.layers.Dense(10, activation='softmax'))

optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5", save_best_only=True)

model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=[early_stopping_cb, model_checkpoint_cb])

model.evaluate(X_valid, y_valid)

Epoch 1/100
1407/1407 [==============================] - 55s 20ms/step - loss: 1.8394 - accuracy: 0.3400 - val_loss: 1.6269 - val_accuracy: 0.4188
Epoch 2/100
1407/1407 [==============================] - 26s 19ms/step - loss: 1.6607 - accuracy: 0.4075 - val_loss: 1.5416 - val_accuracy: 0.4516
Epoch 3/100
1407/1407 [==============================] - 30s 21ms/step - loss: 1.5914 - accuracy: 0.4322 - val_loss: 1.5061 - val_accuracy: 0.4690
Epoch 4/100
1407/1407 [==============================] - 27s 19ms/step - loss: 1.5451 - accuracy: 0.4519 - val_loss: 1.4981 - val_accuracy: 0.4588
Epoch 5/100
1407/1407 [==============================] - 25s 18ms/step - loss: 1.4999 - accuracy: 0.4653 - val_loss: 1.4490 - val_accuracy: 0.4878
Epoch 6/100
1407/1407 [==============================] - 28s 20ms/step - loss: 1.4639 - accuracy: 0.4813 - val_loss: 1.4111 - val_accuracy: 0.5032
Epoch 7/100
1407/1407 [==============================] - 26s 18ms/step - loss: 1.4300 - accuracy: 0.4923 - val_loss: 1

[1.3780597448349, 0.5454000234603882]

In [45]:
# d. 배치 정규화를 SELU로 바꾸기
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
for _ in range(20) :
    model.add(keras.layers.Dense(100, kernel_initializer='lecun_normal', activation='selu'))
model.add(keras.layers.Dense(10, activation='softmax'))

optimizer = keras.optimizers.Nadam(learning_rate=7e-4)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_selu_model.h5", save_best_only=True)

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=[early_stopping_cb, model_checkpoint_cb])

model.evaluate(X_valid_scaled, y_valid)

Epoch 1/100
1407/1407 [==============================] - 27s 12ms/step - loss: 1.9197 - accuracy: 0.3120 - val_loss: 1.8176 - val_accuracy: 0.3574
Epoch 2/100
1407/1407 [==============================] - 20s 14ms/step - loss: 1.7100 - accuracy: 0.3919 - val_loss: 1.6780 - val_accuracy: 0.3970
Epoch 3/100
1407/1407 [==============================] - 18s 13ms/step - loss: 1.6130 - accuracy: 0.4306 - val_loss: 1.6874 - val_accuracy: 0.3994
Epoch 4/100
1407/1407 [==============================] - 16s 11ms/step - loss: 1.5488 - accuracy: 0.4534 - val_loss: 1.6016 - val_accuracy: 0.4170
Epoch 5/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.4905 - accuracy: 0.4763 - val_loss: 1.5455 - val_accuracy: 0.4480
Epoch 6/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.4400 - accuracy: 0.4983 - val_loss: 1.5468 - val_accuracy: 0.4452
Epoch 7/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.4019 - accuracy: 0.5085 - val_loss: 1.5

[1.653214454650879, 0.49900001287460327]

In [46]:
# e. 알파 드롭아웃으로 모델에 규제 적용하고, 모델을 다시 훈련하지 않고 MC 드롭아웃으로 더 높은 정확도 얻기
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
for _ in range(20) :
    model.add(keras.layers.Dense(100, activation='selu', kernel_initializer='lecun_normal'))
model.add(keras.layers.AlphaDropout(0.1))
model.add(keras.layers.Dense(10, activation='softmax'))

optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_alpha_dropout_model.h5", save_best_only=True)

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=[early_stopping_cb, model_checkpoint_cb])

model.evaluate(X_valid_scaled, y_valid)

Epoch 1/100
1407/1407 [==============================] - 24s 11ms/step - loss: 1.8914 - accuracy: 0.3251 - val_loss: 1.7805 - val_accuracy: 0.3908
Epoch 2/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.6550 - accuracy: 0.4130 - val_loss: 1.5948 - val_accuracy: 0.4416
Epoch 3/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.5657 - accuracy: 0.4472 - val_loss: 1.6232 - val_accuracy: 0.4512
Epoch 4/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.5000 - accuracy: 0.4730 - val_loss: 1.5833 - val_accuracy: 0.4582
Epoch 5/100
1407/1407 [==============================] - 16s 11ms/step - loss: 1.4458 - accuracy: 0.4924 - val_loss: 1.5374 - val_accuracy: 0.4646
Epoch 6/100
1407/1407 [==============================] - 18s 13ms/step - loss: 1.4057 - accuracy: 0.5066 - val_loss: 1.5912 - val_accuracy: 0.4748
Epoch 7/100
1407/1407 [==============================] - 20s 14ms/step - loss: 1.3571 - accuracy: 0.5262 - val_loss: 1.

[1.8141587972640991, 0.5027999877929688]

In [47]:
class MCAlphaDropout(keras.layers.AlphaDropout) :
    def call(self, inputs) :
        return super().call(inputs, training=True)

In [48]:
mc_model = keras.models.Sequential([
    MCAlphaDropout(layer.rate) if isinstance(layer, keras.layers.AlphaDropout) else layer
    for layer in model.layers
])

In [49]:
def mc_dropout_predict_probas(mc_model, X, n_samples=10) :
    y_probas = [mc_model.predict(X) for sample in range(n_samples)]
    return np.mean(y_probas, axis=0)

In [50]:
def mc_dropout_predict_classes(mc_model, X, n_samples=10) :
    y_probas = mc_dropout_predict_probas(mc_model, X, n_samples)
    return np.argmax(y_probas, axis=1)

In [51]:
y_pred = mc_dropout_predict_classes(mc_model, X_valid_scaled)
accuracy = np.mean(y_pred==y_valid[:, 0])
accuracy

157/157 [==============================] - 1s 3ms/step


0.502

In [52]:
# f. 1사이클 스케줄링으로 모델훈련
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
for _ in range(20) :
    model.add(keras.layers.Dense(100, kernel_initializer='lecun_normal', activation='selu'))
model.add(keras.layers.AlphaDropout(0.1))
model.add(keras.layers.Dense(10, activation='softmax'))

optimizer=keras.optimizers.SGD(learning_rate=1e-3)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [54]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))

model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.SGD(learning_rate=1e-2)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [61]:
K = keras.backend
class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, rate)

In [62]:
import math
n_epochs = 15
onecycle = OneCycleScheduler(math.ceil(len(X_train_scaled) / batch_size) * n_epochs, max_rate=0.05)
history = model.fit(X_train_scaled, y_train, epochs=n_epochs, batch_size=batch_size,
                    validation_data=(X_valid_scaled, y_valid),
                    callbacks=[onecycle])

Epoch 1/15
352/352 [==============================] - 22s 47ms/step - loss: 2.0686 - accuracy: 0.2837 - val_loss: 1.8017 - val_accuracy: 0.3624
Epoch 2/15
352/352 [==============================] - 10s 29ms/step - loss: 1.7710 - accuracy: 0.3738 - val_loss: 1.6806 - val_accuracy: 0.4054
Epoch 3/15
352/352 [==============================] - 10s 29ms/step - loss: 1.6325 - accuracy: 0.4215 - val_loss: 1.6137 - val_accuracy: 0.4354
Epoch 4/15
352/352 [==============================] - 10s 28ms/step - loss: 1.5534 - accuracy: 0.4494 - val_loss: 1.6588 - val_accuracy: 0.4284
Epoch 5/15
352/352 [==============================] - 10s 28ms/step - loss: 1.4993 - accuracy: 0.4678 - val_loss: 1.6546 - val_accuracy: 0.4256
Epoch 6/15
352/352 [==============================] - 10s 28ms/step - loss: 1.4565 - accuracy: 0.4823 - val_loss: 1.6761 - val_accuracy: 0.4182
Epoch 7/15
352/352 [==============================] - 11s 31ms/step - loss: 1.4148 - accuracy: 0.4964 - val_loss: 1.5526 - val_accuracy: